# Social Media Monitoring

## Politician List


The following table contains data on the politicians and their associated social acounts

In [ ]:
import qgrid
from smm_wrapper import SMM
qgrid.set_grid_option('maxVisibleRows', 5)

# Create an instance to the smm wrapper
smm = SMM()

# Request the politicians from the API
df = smm.dv.get_politicians()

# Show the result
qgrid.show_grid(df)

## Instructions

Please choose an `affiliation` party from the table above and enter its name in the field below:

In [ ]:
party = 'CSU'
aggregation = 'month' #aggregation can be by day ('day'), week ('week') or month('month')

In [ ]:
#aggregating posts and comments by party affiliation
party_fb = {'party':party,'labels':[],'values':[], 'replies':[], 'shares':[], 'reactions':[], 'likes':[],
                'comments':[], 'com_replies':[], 'com_likes':[]}
fb = []
fb_com = []  
for index, row in df.iterrows():
    if row['fb_ids'] is not None and row['affiliation'] == party:
        fb.append(smm.api.posts_by(facebook_user_id=row['fb_ids'][0], aggregate_by=aggregation))
        fb_com.append(smm.api.comments_by(facebook_user_id=row['fb_ids'][0], aggregate_by=aggregation))

In [ ]:
for element in fb:
    [element.pop(key) for key in ['response_type','aggregated_by']]
    for date in element['labels']:
        if date in party_fb['labels']:
            curr_i_fb = party_fb['labels'].index(date)
            curr_i_el = element['labels'].index(date)
            for i,v in element.items():
                if i == 'labels':
                    continue
                else:
                    party_fb[i][curr_i_fb] += v[curr_i_el]
                if i=='likes': break
        else:
            curr_i = element['labels'].index(date)
            for i,v in element.items():
                party_fb[i].append(v[curr_i])

In [ ]:
#SORTING
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

#plot for fb posts
plotly.offline.iplot({
    "data": [go.Scatter(x=party_fb['labels'], y=party_fb['values'], name='Posts'), 
             go.Scatter(x=party_fb['labels'], y=party_fb['replies'], name='Replies'),
            go.Scatter(x=party_fb['labels'], y=party_fb['shares'], name='Shares'),
            go.Scatter(x=party_fb['labels'], y=party_fb['reactions'], name='Reactions'),
            go.Scatter(x=party_fb['labels'], y=party_fb['likes'], name='Likes')], 
    "layout": go.Layout(hovermode='closest', title='Facebook posts', xaxis={'title':'Date'}, yaxis={'title':'N'})
})
#plot for fb comments
#plotly.offline.iplot({
    #"data": [go.Scatter(x=comments['labels'], y=comments['values'], name='Comments'), 
             #go.Scatter(x=comments['labels'], y=comments['replies'], name='Replies'),
            #go.Scatter(x=comments['labels'], y=comments['likes'], name='Likes')], 
    #"layout": go.Layout(hovermode='closest', title='Facebook comments', xaxis={'title':'Date'}, yaxis={'title':'N'})
#})